# Burrow Fear Conditioning Analysis Pipeline

This is a jupyter notebook to conduct the 'standard' analysis of burrow fear conditioning data.

## Experiment Hierarchy

Here we construct and organizational framework for the various behavioral stages and meta-data.

### Import Modules

In [1]:
from AnalysisModules.ExperimentHierarchy import ExperimentData

### Identify Directory, Study, & Mouse Information

In [2]:
__directory="H:\\DEM_Excitatory_Study\\DEM2"
__mouse="M4584"
__study="DEM"
__study_mouse="DEM2"

### Construct Experiment Hierarchy

In [3]:
Data = ExperimentData(Directory=__directory, Mouse=__mouse, Study=__study, StudyMouse=__study_mouse)

## Conduct Analysis on Encoding

### Import Modules

In [4]:
from AnalysisModules.BurrowFearConditioning import Encoding

In [5]:
Data.Encoding = Encoding(Data.passMeta())

### Identify Dataset and Parameters

In [6]:
__data_folder = "H:\\DEM_Excitatory_Study\\DEM2\\Encoding\\Imaging\\10Hz"
__index_file = "H:\\DEM_Excitatory_Study\\DEM2\\Encoding\\Imaging\\10Hz\\NeuronalIndex.csv"
__features_file = "H:\\DEM_Excitatory_Study\\DEM2\\Encoding\\Imaging\\10Hz\\Features.csv"

### Import Modules

In [7]:
# FISSA: Signal Extraction & Source-Separation
from AnalysisModules.FissaAnalysis import FissaModule
# Assorted Static Processing Functions
from AnalysisModules.StaticProcessing import smoothTraces_TiffOrg, calculate_dFoF, mergeTraces, detrendTraces

### Instantiation

In [8]:
# Instantiate Fissa Module & Sub-Modules. 
# Sub-Module 1 is preparation, will contain raw data (PreparationModule)
# Sub-Module 2 is experiment, will contain separation data (SeparationModule)
# Sub-Module 3 is ProcessedTraces, just a container for processed signals
Data.Encoding.Fissa = FissaModule(data_folder=__data_folder, index_file = __index_file)
# This folder contains the suite2p/plane0/___.npy files as well as the saved registered files located in the suite2p/plane0/reg_tif folder
# Initialize
Data.Encoding.Fissa.pruneNonNeuronalROIs() # This step removes all non-neuronal data
Data.Encoding.Fissa.initializeFissa()


Adjusting from 1-to-0 indexing
Initialized Fissa


### Extract Traces

In [9]:
Data.Encoding.Fissa.extractTraces() # simple, call to extract raw traces from videos

Extracting traces:   0%|          | 0/5 [00:00<?, ?it/s]

Finished extracting raw signals from 978 ROIs across 5 trials in 4 min, 20 sec.
Passing traces between modules.
Finished module-passing.
Ready for post-processing or source-separation.


### Save Progress

In [10]:
Data.Encoding.Fissa.saveFissaPrep()

Saving extracted traces to H:\DEM_Excitatory_Study\DEM2\Encoding\Imaging\10Hz\prepared.npz
Finished Saving Prep


### Pre-Process Traces

In [11]:
# let's smooth the data to make it nicer
Data.Encoding.Fissa.ProcessedTraces.smoothed_raw = smoothTraces_TiffOrg(Data.Encoding.Fissa.preparation.raw, niter=50, kappa=150, gamma=0.15)[0]

Smoothing:   0%|          | 0/978 [00:00<?, ?it/s]

Organizing By Tiff:   0%|          | 0/978 [00:00<?, ?it/s]

### Save Progress

In [12]:
Data.Encoding.Fissa.saveProcessedTraces()

Saving Processed Traces...
Finished Saving Processed Traces.


### Use Pre-Processed Traces for Source-Separation

In [13]:
Data.Encoding.Fissa.preparation.raw = Data.Encoding.Fissa.ProcessedTraces.smoothed_raw.copy()
#Let's use for separation, so replace the raws with smooths
Data.Encoding.Fissa.passPrepToFissa()

### Source-Separate Traces

In [14]:
Data.Encoding.Fissa.separateTraces() # simple, call to separate the traces

Passing prepared traces to fissa.
Initiating fissa source-separation


Separating data:   0%|          | 0/978 [00:00<?, ?it/s]

Finished separating signals from 978 ROIs across 5 trials in 1:13:25.841923
Passing traces between modules.
Finished module-passing.
Ready for further analysis.


### Save Progress

In [15]:
Data.Encoding.Fissa.saveFissaSep()

Saving results to H:\DEM_Excitatory_Study\DEM2\Encoding\Imaging\10Hz\separated.npz


### Post-Process Traces

In [16]:
# Calculate Fo/F
Data.Encoding.Fissa.ProcessedTraces.dFoF_result = calculate_dFoF(Data.Encoding.Fissa.experiment.result, Data.Encoding.Fissa.frame_rate, raw=Data.Encoding.Fissa.preparation.raw, merge_after=False)
# Condense the ROI Traces for each Trial into a Single Matrix
Data.Encoding.Fissa.ProcessedTraces.merged_dFoF_result = mergeTraces(Data.Encoding.Fissa.ProcessedTraces.dFoF_result)
# Detrend the Traces by fitting a 4th-order polynomial and subsequently subtracting
Data.Encoding.Fissa.ProcessedTraces.detrended_merged_dFoF_result = detrendTraces(Data.Encoding.Fissa.ProcessedTraces.merged_dFoF_result, order=4, plot=False)

Calculating Δf/f0  (using single f0 across all tiffs using f0 derived from raw traces during calculations)


Calculating Δf/f0:   0%|          | 0/978 [00:00<?, ?it/s]

Merging Traces Across Tiffs:   0%|          | 0/978 [00:00<?, ?it/s]

Detrending:   0%|          | 0/978 [00:00<?, ?it/s]

### Save Progress

In [17]:
Data.Encoding.Fissa.saveProcessedTraces()

Saving Processed Traces...
Finished Saving Processed Traces.


## Infer Firing Rates & Approximate Spikes

### Import Modules

In [18]:
# CASCADE: Spike Inference
from AnalysisModules.CascadeAnalysis import CascadeModule
# Assorted Static Processing Functions
from AnalysisModules.StaticProcessing import calculateFiringRate
# Assorted Static Utility Functions
from AnalysisModules.StaticUtilities import pullModels

C:\ProgramData\Anaconda3\envs\Calcium-Imaging-Analysis-Pipeline\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


	YAML reader installed (version 0.17.17).


Using TensorFlow backend.


	Keras installed (version 2.3.1).
	Tensorflow installed (version 2.3.0).


If returning, use this cell to load prior data

In [19]:
# Instantiate
#Data = FissaModule(data_folder="H:\\DEM_Excitatory_Study\\DEM2\\Retrieval\\Imaging\\10Hz", index_file="H:\\DEM_Excitatory_Study\\DEM2\\Retrieval\\Imaging\\10Hz\\NeuronalIndex.csv")
# Initialize
#Data.initializeFissa()
# Load
#Data.loadFissaPrep()
#Data.loadFissaSep()
#Data.loadProcessedTraces()

### Instantiation

In [20]:
# Pass the Post-Processed Traces. Here we choose the detrended, matrix-form, Fo/F traces
Data.Encoding.Cascade = CascadeModule(Data.Encoding.Fissa.ProcessedTraces.detrended_merged_dFoF_result, Data.Encoding.Fissa.frame_rate, model_folder="C:\\ProgramData\\Anaconda3\\envs\\suite2p\\Pretrained_models")

### Model Selection

In [21]:
# Pull Available Models
list_of_models = pullModels(Data.Encoding.Cascade.model_folder)

# Select Model: If you know what model you want, you should use the string instead.
# This model is Global_EXC_10Hz_smoothing_100ms
Data.Encoding.Cascade.model_name = list_of_models[21]

# Download Model: You will need to download the model before first use
# from AnalysisModules.StaticUtilities import downloadModel

You can now check the updated available_models.yaml file for valid model names.
File location: C:\ProgramData\Anaconda3\envs\suite2p\Pretrained_models\available_models.yaml

 List of available models: 

Global_EXC_30Hz_smoothing50ms_asymmetric_window_1_frame
Global_EXC_30Hz_smoothing50ms_asymmetric_window_2_frames
Global_EXC_30Hz_smoothing50ms_asymmetric_window_4_frames
Global_EXC_30Hz_smoothing50ms_asymmetric_window_6_frames
Global_EXC_30Hz_smoothing50ms_asymmetric_window_8_frames
GCaMP6f_mouse_30Hz_smoothing200ms
OGB_zf_pDp_7.5Hz_smoothing200ms
OGB_zf_pDp_7.5Hz_smoothing200ms_causalkernel
Global_EXC_3Hz_smoothing400ms
Global_EXC_3Hz_smoothing400ms_high_noise
Global_EXC_3Hz_smoothing400ms_causalkernel
Global_EXC_4.25Hz_smoothing300ms
Global_EXC_4.25Hz_smoothing300ms_high_noise
Global_EXC_4.25Hz_smoothing300ms_causalkernel
Global_EXC_5Hz_smoothing200ms
Global_EXC_5Hz_smoothing200ms_causalkernel
Global_EXC_6Hz_smoothing200ms
Global_EXC_6Hz_smoothing200ms_causalkernel
Global_EXC_7.5Hz_sm

In [24]:
Data.Encoding.Cascade.model_name = "Global_EXC_10Hz_smoothing100ms"

### Infer Firing Rates

In [25]:
# Infer Spike Probability
Data.Encoding.Cascade.predictSpikeProb() # Simple, call to infer spike probability for each frame
# Calculate Firing Rates # Simple, firing rate = spike probability * imaging frequency
Data.Encoding.Cascade.ProcessedInferences.firing_rates = calculateFiringRate(Data.Encoding.Cascade.spike_prob, Data.Encoding.Cascade.frame_rate)


 
The selected model was trained on 18 datasets, with 5 ensembles for each noise level, at a sampling rate of 10Hz, with a resampled ground truth that was smoothed with a Gaussian kernel of a standard deviation of 100 milliseconds. 
 

Loaded model was trained at frame rate 10 Hz
Given argument traces contains 978 neurons and 33551 frames.
Noise levels (mean, std; in standard units): 0.07, 0.04

Predictions for noise level 2:
	... ensemble 0
4006/4006 [==============================] - 18s 4ms/step
	... ensemble 1
4006/4006 [==============================] - 18s 4ms/step
	... ensemble 2
4006/4006 [==============================] - 19s 5ms/step
	... ensemble 3
4006/4006 [==============================] - 18s 4ms/step
	... ensemble 4
4006/4006 [==============================] - 18s 4ms/step

Predictions for noise level 3:
	No neurons for this noise level

Predictions for noise level 4:
	No neurons for this noise level

Predictions for noise level 5:
	No neurons for this noise level

Pre

### Save Progress

In [26]:
Data.Encoding.Cascade.saveSpikeProb(Data.Encoding.Fissa.output_folder)
Data.Encoding.Cascade.saveProcessedInferences(Data.Encoding.Fissa.output_folder)

Saving Processed Inferences...
Finished Saving Processed Inferences


### Infer Discrete Spike Times

In [27]:

# Infer the discrete spike events, simply call
Data.Encoding.Cascade.inferDiscreteSpikes()

# If you have an infinite loop:
# you are looking at a neuron that should have been dropped for
# (1) exceedingly poor SNR
# (2) major trend in the data (e.g., lots of bleaching)
# (3) abnormal phenotype for particular model-- think of training your own!

Infer spikes for neuron 1 out of 978
Infer spikes for neuron 2 out of 978
Infer spikes for neuron 3 out of 978
Infer spikes for neuron 4 out of 978
Infer spikes for neuron 5 out of 978
Infer spikes for neuron 6 out of 978
Infer spikes for neuron 7 out of 978
Infer spikes for neuron 8 out of 978
Infer spikes for neuron 9 out of 978
Infer spikes for neuron 10 out of 978
Infer spikes for neuron 11 out of 978
Infer spikes for neuron 12 out of 978
Infer spikes for neuron 13 out of 978
Infer spikes for neuron 14 out of 978
Infer spikes for neuron 15 out of 978
Infer spikes for neuron 16 out of 978
Infer spikes for neuron 17 out of 978
Infer spikes for neuron 18 out of 978
Infer spikes for neuron 19 out of 978
Infer spikes for neuron 20 out of 978
Infer spikes for neuron 21 out of 978
Infer spikes for neuron 22 out of 978
Infer spikes for neuron 23 out of 978
Infer spikes for neuron 24 out of 978
Infer spikes for neuron 25 out of 978
Infer spikes for neuron 26 out of 978
Infer spikes for neur

Infer spikes for neuron 215 out of 978
Infer spikes for neuron 216 out of 978
Infer spikes for neuron 217 out of 978
Infer spikes for neuron 218 out of 978
Infer spikes for neuron 219 out of 978
Infer spikes for neuron 220 out of 978
Infer spikes for neuron 221 out of 978
Infer spikes for neuron 222 out of 978
Infer spikes for neuron 223 out of 978
Infer spikes for neuron 224 out of 978
Infer spikes for neuron 225 out of 978
Infer spikes for neuron 226 out of 978
Infer spikes for neuron 227 out of 978
Infer spikes for neuron 228 out of 978
Infer spikes for neuron 229 out of 978
Infer spikes for neuron 230 out of 978
Infer spikes for neuron 231 out of 978
Infer spikes for neuron 232 out of 978
Infer spikes for neuron 233 out of 978
Infer spikes for neuron 234 out of 978
Infer spikes for neuron 235 out of 978
Infer spikes for neuron 236 out of 978
Infer spikes for neuron 237 out of 978
Infer spikes for neuron 238 out of 978
Infer spikes for neuron 239 out of 978
Infer spikes for neuron 2

Infer spikes for neuron 426 out of 978
Infer spikes for neuron 427 out of 978
Infer spikes for neuron 428 out of 978
Infer spikes for neuron 429 out of 978
Infer spikes for neuron 430 out of 978
Infer spikes for neuron 431 out of 978
Infer spikes for neuron 432 out of 978
Infer spikes for neuron 433 out of 978
Infer spikes for neuron 434 out of 978
Infer spikes for neuron 435 out of 978
Infer spikes for neuron 436 out of 978
Infer spikes for neuron 437 out of 978
Infer spikes for neuron 438 out of 978
Infer spikes for neuron 439 out of 978
Infer spikes for neuron 440 out of 978
Infer spikes for neuron 441 out of 978
Infer spikes for neuron 442 out of 978
Infer spikes for neuron 443 out of 978
Infer spikes for neuron 444 out of 978
Infer spikes for neuron 445 out of 978
Infer spikes for neuron 446 out of 978
Infer spikes for neuron 447 out of 978
Infer spikes for neuron 448 out of 978
Infer spikes for neuron 449 out of 978
Infer spikes for neuron 450 out of 978
Infer spikes for neuron 4

Infer spikes for neuron 637 out of 978
Infer spikes for neuron 638 out of 978
Infer spikes for neuron 639 out of 978
Infer spikes for neuron 640 out of 978
Infer spikes for neuron 641 out of 978
Infer spikes for neuron 642 out of 978
Infer spikes for neuron 643 out of 978
Infer spikes for neuron 644 out of 978
Infer spikes for neuron 645 out of 978
Infer spikes for neuron 646 out of 978
Infer spikes for neuron 647 out of 978
Infer spikes for neuron 648 out of 978
Infer spikes for neuron 649 out of 978
Infer spikes for neuron 650 out of 978
Infer spikes for neuron 651 out of 978
Infer spikes for neuron 652 out of 978
Infer spikes for neuron 653 out of 978
Infer spikes for neuron 654 out of 978
Infer spikes for neuron 655 out of 978
Infer spikes for neuron 656 out of 978
Infer spikes for neuron 657 out of 978
Infer spikes for neuron 658 out of 978
Infer spikes for neuron 659 out of 978
Infer spikes for neuron 660 out of 978
Infer spikes for neuron 661 out of 978
Infer spikes for neuron 6

Infer spikes for neuron 848 out of 978
Infer spikes for neuron 849 out of 978
Infer spikes for neuron 850 out of 978
Infer spikes for neuron 851 out of 978
Infer spikes for neuron 852 out of 978
Infer spikes for neuron 853 out of 978
Infer spikes for neuron 854 out of 978
Infer spikes for neuron 855 out of 978
Infer spikes for neuron 856 out of 978
Infer spikes for neuron 857 out of 978
Infer spikes for neuron 858 out of 978
Infer spikes for neuron 859 out of 978
Infer spikes for neuron 860 out of 978
Infer spikes for neuron 861 out of 978
Infer spikes for neuron 862 out of 978
Infer spikes for neuron 863 out of 978
Infer spikes for neuron 864 out of 978
Infer spikes for neuron 865 out of 978
Infer spikes for neuron 866 out of 978
Infer spikes for neuron 867 out of 978
Infer spikes for neuron 868 out of 978
Infer spikes for neuron 869 out of 978
Infer spikes for neuron 870 out of 978
Infer spikes for neuron 871 out of 978
Infer spikes for neuron 872 out of 978
Infer spikes for neuron 8

### Save Progress

In [28]:
Data.Encoding.Cascade.saveSpikeInference(Data.Encoding.Fissa.output_folder)

C:\ProgramData\Anaconda3\envs\Calcium-Imaging-Analysis-Pipeline\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


### Export to MATLab for MATLab Analyses

In [29]:
# Here, we can export the spike prob or events to matlab .mat files
Data.Cascade.exportSpikeProb(Data.Encoding.Fissa.output_folder)
Data.Cascade.exportSpikeInference(Data.Encoding.Fissa.output_folder)

C:\Users\YUSTE\AppData\Roaming\Python\Python37\site-packages\scipy\io\matlab\mio5.py:450: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  narr = np.asanyarray(source)


# Visualization

### Assess Inference of Spike Probability & Approximation of Discrete Spikes

In [30]:
from AnalysisModules.StaticPlotting import assessSpikeInference
assessSpikeInference(Data.Encoding.Cascade.spike_prob, Data.Encoding.Cascade.spike_time_estimates, Data.Encoding.Cascade.traces, Data.Encoding.Cascade.frame_rate)

right


Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\Calcium-Imaging-Analysis-Pipeline\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    func(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\Calcium-Imaging-Analysis-Pipeline\AnalysisModules\StaticPlotting.py", line 147, in on_key
    if xmin_slider < xmin_slider.valmax:
TypeError: '>' not supported between instances of 'float' and 'Slider'


right


Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\Calcium-Imaging-Analysis-Pipeline\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    func(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\Calcium-Imaging-Analysis-Pipeline\AnalysisModules\StaticPlotting.py", line 147, in on_key
    if xmin_slider < xmin_slider.valmax:
TypeError: '>' not supported between instances of 'float' and 'Slider'


In [31]:
Data.Encoding.saveHierarchy()

AttributeError: 'FissaModule' object has no attribute 'saveHierarchy'